In [1]:
import pandas as pd
import plotly.graph_objects as go
import requests
from tqdm.auto import tqdm

In [2]:
def sunshine_in_month(month: str):
    data = fetch_data(f"https://www.currentresults.com/Weather/Europe/Cities/sunshine-average-{month}.php")
    return data.rename({"Hours": "sunshine"}, axis="columns") / 30

def fetch_data(url: str):
    html = requests.get(url).text
    tables = pd.read_html(html)
    concatenated = pd.concat(tables, ignore_index=True)
    split = concatenated["City"].str.split(", ", expand=True).rename({0: "city", 1: "country"}, axis="columns")
    joined = pd.concat([concatenated.drop("City", axis="columns"), split], axis="columns")
    return joined.set_index(["country", "city"])

sunshine_in_month("january").loc["Italy"]

,sunshine
city,
Milan,1.966667
Naples,3.833333
Turin,3.733333


In [3]:
def climate_in_month(month: str):
    return pd.concat([sunshine_in_month(month), temperature_in_month(month)], axis="columns")

def temperature_in_month(month: str):
    full_data = fetch_data(f"https://www.currentresults.com/Weather/Europe/Cities/temperature-{month}.php")
    return full_data.drop(["High °F", "Low °F", "Low °C"], axis="columns").rename({"High °C": "temperature"}, axis="columns")

climate_in_month("january").loc["Italy"]

,sunshine,temperature
city,,
Milan,1.966667,6.0
Naples,3.833333,13.0
Turin,3.733333,7.0
Rome,NaN,13.0


In [4]:
all_months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
climate_data = pd.concat(
    {month: climate_in_month(month) for month in tqdm(all_months)},
    axis="columns"
).dropna()
climate_data.xs("sunshine", axis="columns", level=1).loc["France"]

  0%|          | 0/12 [00:00<?, ?it/s]

,january,february,march,april,may,june,july,august,september,october,november,december
city,,,,,,,,,,,,
Lille,2.200000,2.366667,4.033333,5.733333,6.466667,6.866667,7.033333,6.666667,5.066667,3.800000,2.033333,1.666667
Lyon,2.466667,3.366667,5.666667,6.366667,7.366667,8.466667,9.433333,8.433333,6.500000,4.333333,2.533333,1.800000
Marseille,4.833333,5.800000,7.966667,8.133333,9.766667,11.100000,12.300000,10.900000,8.633333,6.233333,5.066667,4.500000
Nice,5.266667,5.700000,7.233333,7.466667,8.900000,10.200000,11.600000,10.533333,8.066667,6.233333,4.966667,4.633333
Paris,2.066667,2.633333,4.300000,5.533333,6.466667,6.733333,7.066667,7.066667,5.600000,3.933333,2.266667,1.700000


In [5]:
aggregate_data = pd.concat(
    {
        "least sunshine": climate_data.xs("sunshine", axis="columns", level=1).min("columns"),
        "low temperature": climate_data.xs("temperature", axis="columns", level=1).min("columns"),
        "high temperature": climate_data.xs("temperature", axis="columns", level=1).max("columns"),
    },
    axis="columns"
).sort_values("least sunshine")
aggregate_data.loc["Spain"]

,least sunshine,low temperature,high temperature
city,,,
Madrid,4.133333,10.0,31.0
Barcelona,4.600000,13.0,28.0


In [12]:
go.Figure(
    layout=dict(
        yaxis=dict(
            visible=False
        ),
        xaxis_title="Range of max daily temperature",
        width=1024,
        height=2048,
        template="plotly_dark",
    ),
    data=[
        go.Bar(
            orientation="h",
            base=aggregate_data["low temperature"],
            x=aggregate_data["high temperature"],
            text=aggregate_data.index.get_level_values(1),
            textposition="inside",
            insidetextanchor="middle",
            marker=dict(
                color=aggregate_data["least sunshine"],
                showscale=True,
                colorbar=dict(
                    title=dict(
                        text="Daily hours of sun in darkest month",
                        side="top",
                    ),
                    orientation="h",
                    y=-0.08,
                )
            ),
        )
    ]
)